In [2]:
import networkx as nx
from fics import TypedFunction
from fics import FiniteInverseCategory, FICStructure, FICStructureHomomorphism

# Demonstration of the fics implementation of finite inverse categories

## Finite Inverse Categories

Let us encode the simple example of $\mathcal{L}_{\text{rg}}$ to get us started.

$\mathcal{L}_{\text{rg}}$ is the finite inverse category with:
* objects $O,A,I$
* arrows $d,c : A \rightarrow O$, $i: I \rightarrow A$
* relations $id = ic$ (in diagrammatic order of compositions)

In [3]:
G = nx.MultiDiGraph()
G.add_edges_from([('A','O','d'),('A','O','c'),('I','A','i')])
Lrg = FiniteInverseCategory(G,[(('I','O','id'),('I','O','ic'))])

To view the set of morphisms $\mathcal{L}_{\text{rg}} (I,O)$ one does this...

In [4]:
Lrg.Hom_e[('I','O')]

[('I', 'O', 'id')]

...and to view the set of all morphisms (i.e. even distinct equal ones), which is occasionally useful, one does this:

In [5]:
Lrg.Hom[('I','O')]

[('I', 'O', 'id'), ('I', 'O', 'ic')]

To get the domain and the codomain of a morphism:

In [6]:
assert Lrg.cod['i'] == 'A'
assert Lrg.dom['id'] == 'I'

To interpret $\mathcal{L}_{\text{rg}}$ as a $\Sigma$-type in UniMath:

In [7]:
Lrg.UniMathDataShape()

## FIC structures

Now let us define an $\mathcal{L}_{\text{rg}}$-structure $M$, i.e. a functor $M: \mathcal{L}_{\text{rg}} \rightarrow \textbf{Set}$

In [8]:
obval = {'O':['x','y'], 'A':['p'],'I':['s']}
morval = {('A','O','d'):TypedFunction(obval['A'],obval['O'],{'p':'x'}),\
          ('A','O','c'):TypedFunction(obval['A'],obval['O'],{'p':'x'}),\
          ('I','A','i'):TypedFunction(obval['I'],obval['A'],{'s':'p'})}

In [9]:
M = FICStructure(Lrg,obval,morval)

To check the value of the functor on objects do:

In [10]:
M.obval

{'O': ['x', 'y'], 'A': ['p'], 'I': ['s']}

And to interpret $M$ in UniMath do:

In [11]:
M.UniMathDataPoint()

Let us also define the yoneda $\mathcal{L}_{\text{rg}}$-structure $\textbf{y}I$ and the reduced yoneda $\mathcal{L}_{\text{rg}}$-structure $\partial I$

In [12]:
Y = M
Y = M.yoneda('I')
Yhat = M.reducedyoneda('I')

In [13]:
Y.obval

{'A': [('I', 'A', 'i')], 'O': [('I', 'O', 'id')], 'I': ['id_I']}

In [14]:
Yhat.obval

{'A': [('I', 'A', 'i')], 'O': [('I', 'O', 'id')], 'I': []}

## FIC Transformations

Let us define a natural transformation $\alpha : \partial I \to \mathcal{L}_{\text{rg}}$

In [15]:
YhatO = Yhat.obval['O']
MO = M.obval['O']
YhatA = Yhat.obval['A']
MA = M.obval['A']
# the functions corresponding to the data for the natural transformation
a_O = TypedFunction(YhatO,MO,{('I', 'O', 'id'):'x'})
a_A = TypedFunction(YhatA,MA,{('I', 'A', 'i'):'p'})
# definiing the FICStructureHomomorphism object
alpha = FICStructureHomomorphism(Lrg,Yhat, M,{'O':a_O,'A':a_A})

In [16]:
alpha.funcval['O'].app

{('I', 'O', 'id'): 'x'}

## A "Data Shape" toy example

Now for a slightly more realistic toy example

In [17]:
G = nx.MultiDiGraph()
G.add_edges_from([('Age','Team','a'),('Gender','Team','g'),('Name','Gender','t1'),
                  ('Name','Age','t2')])
L = FiniteInverseCategory(G)

In [18]:
namelist = ['John','Bob','Jane','Pia']
agelist = ['x_'+str(n) for n in range(1,100)]
genderlist = ['M','F','NA']
teamlist = ['Bucks','Bulls']

a = {}
for n in range(1,51):
        a['x_'+str(n)] = 'Bucks'
for n in range(51,100):
        a['x_'+str(n)] = 'Bulls'
a = TypedFunction(agelist,teamlist,a)
g = TypedFunction(genderlist,teamlist,{'M':'Bucks','F':'Bulls','NA':'Bulls'})
t1 = TypedFunction(namelist,genderlist,{'John':'M','Jane':'F','Pia':'NA','Bob':'M'})
t2 = TypedFunction(namelist,agelist,{'John':'x_26','Jane':'x_28','Pia':'x_22','Bob':'x_68'})

In [19]:
obdata = {'Name':namelist,'Age':agelist, 'Gender':genderlist,'Team':teamlist}
mordata = {('Age', 'Team', 'a'):a,('Gender', 'Team', 'g'):g,
           ('Name', 'Gender', 't1'):t1,('Name', 'Age', 't2'):t2}
M = FICStructure(L,obdata,mordata)

In [20]:
M.morval[('Name','Team','t2a')].app['John']

'Bucks'

In [21]:
C_Name = M 
C = C_Name.yoneda('Name')

In [22]:
r_Name = TypedFunction(C.obval['Name'],M.obval['Name'],{'id_Name':'John'})
r_Age = TypedFunction(C.obval['Age'],M.obval['Age'],{('Name', 'Age', 't2'):'x_26'})
r_Gender = TypedFunction(C.obval['Gender'],M.obval['Gender'],{('Name', 'Gender', 't1'):'M'})
r_Team = TypedFunction(C.obval['Team'],M.obval['Team'],{('Name', 'Team', 't1g'):'Bucks',
                                                        ('Name', 'Team', 't2a'):'Bucks'})

In [23]:
row = FICStructureHomomorphism(L,C,M,{'Name':r_Name,'Age':r_Age,'Gender':r_Gender,
                                      'Team':r_Team})

In [24]:
row.funcval['Name'].app['id_Name']

'John'

The `row` is so-called because it is, as I have suggested, the replacement for the concept of a "row" in the paradigm of tabular data.

Thus, for example, the "rows" of a data shape are simply a class of certain natural transformations.

In any case let us write a "data shape" version of the toy "database" in UniMath.

In [25]:
M.UniMathDataPoint('Teams_datashape')